In [1]:
import pandas as pd
import cleaningtools as ct 
import file_config as fconfig
import merge_config as config
import assay_sample_config as aconfig
import numpy as np
import math
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()
from tqdm import tqdm
import os
import file_config as fconfig
import pandas as pd

check for pandas
pandas is properly installed
check for tqdm
tqdm is properly installed
check for numpy
numpy is properly installed


In [2]:
def clean_for_merge(data):
    for col in data.columns:
        if 'depth_ft'in col.lower():
            data['from_ft']=data[col]
            data['to_ft']=data['from_ft'].shift(-1)-1
            #convert ft to meters
            data['from_m']=data['from_ft']* .3281
            data['to_m']=data['to_ft'] * .3281
            data=data.drop(data.filter(like='Depth').columns,axis=1)
        if ('hole' in col.lower())& ('id' in col.lower()):
            data[col]=data[col].astype(object)
            data.rename(columns={f'{col}':f'hole_id'},inplace=True)
        try: 
            data[col]=data[col].str.strip()
        except Exception as e:
            pass
            #print(f'{col}: {e}')
        data.rename(columns={f'{col}':f'{col.strip().lower()}'},inplace=True)
        data=data.drop(data.filter(like='unnamed'),axis=1)
        if 'from_ft'in col.lower():
            try:
                drop_index=data[data['from_ft'].isna()].index
                data=data.drop(drop_index,axis=0)
            except Exception as e:
                pass
                #print(f'{e}')
    if 'hole_id' not in data.columns:
        data['hole_id']=np.nan
    if 'recvd wt.'  in data.columns:
        try:
            data['recvd wt.']=pd.to_numeric(data['recvd wt.'],errors='coerce')
            data['from_ft']=pd.to_numeric(data['from_ft'],errors='coerce')
        except Exception as e:
            print(e)
            pass
    names=data.loc[:,data.columns.duplicated()].columns
    if len(names)>0:
        print(f'duplcated column: {names} drop or else the nasty merge bug')
        data=data.loc[:,~data.columns.duplicated()].copy()

    return data

In [3]:
def fill_merge_groups(data):
    data=data.fillna(method='ffill')
    data=data.fillna(method='bfill')
    data=data.drop_duplicates(keep='first')
    return data

In [4]:
def explode_depths(data):
    data[['from_ft','to_ft']]=data[['from_ft','to_ft']].fillna(0)
    if 'sample_id' in data.columns:
        zipped=zip(data['sample_id'],data['hole_id'], data['from_ft'], data['to_ft'])
        depths=pd.DataFrame([(s_id,h_id, y) for s_id,h_id, start, end in zipped for y in np.arange(start, end,.5)],
                    columns=['sample_id','hole_id','true_depth'])
        d_frame=depths.merge(data,on=['sample_id','hole_id'])
        d_frame=depths.merge(data,on=['sample_id','hole_id'])
    else:
        data['START']=data['from_ft']
        zipped=zip(data['START'],data['hole_id'], data['from_ft'], data['to_ft'])
        depths=pd.DataFrame([(s,h_id, y) for s,h_id, start, end in zipped for y in np.arange(start, end,.5)],
                        columns=['START','hole_id','true_depth'])
        d_frame=depths.merge(data,on=['START','hole_id'])
        d_frame=d_frame.drop('START',axis=1)
    d_frame=d_frame.drop(d_frame.filter(like='from').columns,axis=1)
    d_frame=d_frame.drop(d_frame.filter(like='to').columns,axis=1)
    return d_frame

In [54]:
def pull_start_end(data):
    depths=data['true_depth']
    start=depths.values[0]
    end=depths.values[-1]
    data['from_ft']=start
    data['to_ft']=end
    data=data.drop_duplicates(subset=['from_ft','to_ft'],keep='first')
    return data

In [6]:
hole_list=[]
sample_list=[]
sample_data_names=[]
hole_data_names=[]
files=[file for file in os.listdir(fconfig.output_path) if file.endswith('.csv')]
for file in files:
    name=file.split('.')[0]
    if len(name)==0:
        continue
    print(f'read {name}')
    try:
        data=pd.read_csv(fconfig.output_path+file,low_memory=False,on_bad_lines='skip')

    except Exception as e:
        print(e)
        print('likely not a csv')
        continue
    print(f'clean {name}')
    data=clean_for_merge(data)

    if 'sample_id' in data.columns:
        print(f'{name} has sample ids')
        sample_list.append(data)
        sample_data_names.append(name)
    else:
        print(f'{name} does not have sample ids')
        print(f'explode {name}')
        try:
            data=explode_depths(data)
            hole_list.append(data)
            hole_data_names.append(name)
        except Exception as e:
            print(e)
            print('must have (depths and hole ids) or (sample_ids) to merge into master')

read alteration master
clean alteration master
alteration master does not have sample ids
explode alteration master
read mineralization master
clean mineralization master
duplcated column: Index(['min_type'], dtype='object') drop or else the nasty merge bug
mineralization master does not have sample ids
explode mineralization master
read hyp-package samples master
clean hyp-package samples master
hyp-package samples master has sample ids
read structure master
clean structure master
structure master does not have sample ids
explode structure master
read xrf samples master
clean xrf samples master
xrf samples master has sample ids
read drill assay samples master
clean drill assay samples master
drill assay samples master has sample ids
read drill assays master
clean drill assays master
'from_ft'
drill assays master has sample ids
read lithology master
clean lithology master
lithology master does not have sample ids
explode lithology master
read spectral master
clean spectral master
'from

In [7]:
# merge the data in each group. holes and samples
def merge_dflist(data_list,data_names):
    big_df=pd.DataFrame()
    non_id_cols=['true_depth', 'hole_id']
    for i,data in enumerate(data_list):
        ## merge all the data with sample_ids
        name=data_names[i]
        suf=name.split(' ')[0]
        print(f"####### merge frame {i}: {name} suffix= {suf} #######")
        data=data.astype(object)
        big_df=big_df.astype(object)
        big_cols=list(big_df.columns)
        small_cols=list(data.columns)
        
        dups=sorted([col for col in big_cols if col in small_cols])
        cols=config.main_columns
        try:
            [dups.remove(x) for x in ['file','folder','start_depth','end_depth']if x in data.columns]
        except:
            pass
        if 'sample_id' not in data.columns:
            cols=non_id_cols
            try:
                big_index=list(big_df.hole_id.unique())
                small_index=list(data.hole_id.unique())
                same_ids= [id for id in big_index if id in small_index]
            except Exception as e:
                print(e)
                same_ids=[]
            dups=non_id_cols
        else:
            big_index=list(big_df.index)
            small_index=list(data.index)
            same_ids= [id for id in big_index if id in small_index]
        print(f'common columns {dups}')
        n_ids=len(same_ids)
        if n_ids==0:
            print(f'{n_ids} common ids concat')
            big_df=pd.concat([big_df,data],axis=0,join='outer')
        else:
            print(f'{n_ids} common ids merge')
            big_df=big_df.merge(data,on=dups,suffixes=[None,'_'+suf],how='outer')

        print(f'data shape {big_df.shape}')
    
    not_dups=[i for i in big_df.columns if i not in cols]
    big_final=pd.concat([big_df[cols],big_df[not_dups]],axis=1)

    big_final.drop(big_final.filter(like='file').columns,axis=1,inplace=True)
    big_final.drop(big_final.filter(like='folder').columns,axis=1,inplace=True)
    big_final.sort_index(inplace=True)
    return big_final

In [8]:

big_samples=merge_dflist(sample_list,sample_data_names)
print('FINISHED MERGING SAMPLES')
big_holes=merge_dflist(hole_list,hole_data_names)

####### merge frame 0: hyp-package samples master suffix= hyp-package #######
common columns []
0 common ids concat
data shape (1080, 9)
####### merge frame 1: xrf samples master suffix= xrf #######
common columns ['hole_id', 'sample_id']
432 common ids merge
data shape (1512, 186)
####### merge frame 2: drill assay samples master suffix= drill #######
common columns ['from_ft', 'from_m', 'geo', 'hole_id', 'qaqc', 'sample_id', 'to_ft', 'to_m']
1509 common ids merge
data shape (3103, 194)
####### merge frame 3: drill assays master suffix= drill #######
common columns ['hole_id', 'sample_id']
3103 common ids merge
data shape (8514, 254)
####### merge frame 4: spectral master suffix= spectral #######


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


common columns ['hole_id', 'recvd wt.', 'sample_id']
8044 common ids merge


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


data shape (16080, 377)
FINISHED MERGING SAMPLES
####### merge frame 0: alteration master suffix= alteration #######
'DataFrame' object has no attribute 'hole_id'
common columns ['true_depth', 'hole_id']
0 common ids concat
data shape (109179, 13)
####### merge frame 1: mineralization master suffix= mineralization #######
common columns ['true_depth', 'hole_id']
62 common ids merge
data shape (118186, 24)
####### merge frame 2: structure master suffix= structure #######
common columns ['true_depth', 'hole_id']
82 common ids merge


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


data shape (145555, 32)
####### merge frame 3: lithology master suffix= lithology #######
common columns ['true_depth', 'hole_id']
89 common ids merge


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


data shape (360906, 40)


In [9]:
print('#####clean the sample_ids#########')
big_samples=big_samples.groupby(['sample_id']).progress_apply(fill_merge_groups).droplevel(level=0)
print('#####explode sample_ids#########')
samp_ex=explode_depths(big_samples)
print('#########merge holes and samples##########')
merged=samp_ex.merge(big_holes,on=['hole_id','true_depth'],how='outer')
print('clean the final_data')


#####clean the sample_ids#########


  0%|          | 0/9377 [00:00<?, ?it/s]

#####explode sample_ids#########
#########merge holes and samples##########
clean the final_data


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [10]:
samp_ex.filter(like='depth').isna().sum()

true_depth                  0
depth                   50594
start_depth             50594
end_depth               50594
start_depth_drill       50594
end_depth_drill         50594
start_depth_spectral    19906
end_depth_spectral      19906
dtype: int64

In [11]:
dups=merged[merged.drop('true_depth',axis=1).duplicated(keep=False)]
dups

,sample_id,hole_id,true_depth,geo_x,instrument serial num,reading #,date,time,method name,test label,...,angle,description_structure,"/.,",118,geo_lithology,lithology,texture,description_lithology,fol,geo_2
0,B300001,KM-20-01,884.0,DL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Fault zone. Strongly fractured and locally cru...,NaN,NaN,DL,hwfs,fol,Quartz-sericite schist. Silvery-white. Trace py.,NaN,NaN
1,B300001,KM-20-01,884.5,DL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Fault zone. Strongly fractured and locally cru...,NaN,NaN,DL,hwfs,fol,Quartz-sericite schist. Silvery-white. Trace py.,NaN,NaN
2,B300001,KM-20-01,885.0,DL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Fault zone. Strongly fractured and locally cru...,NaN,NaN,DL,hwfs,fol,Quartz-sericite schist. Silvery-white. Trace py.,NaN,NaN
4,B300001,KM-20-01,885.5,DL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Fault zone. Strongly fractured and locally cru...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,B300001,KM-20-01,886.0,DL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Fault zone. Strongly fractured and locally cru...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370214,NaN,KM-21-33,551.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,RH,hwfs,fol,sericite-quartz schist; light gray; strong fol...,NaN,NaN
370215,NaN,KM-21-33,552.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,RH,hwfs,fol,sericite-quartz schist; light gray; strong fol...,NaN,NaN
370216,NaN,KM-21-33,552.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,RH,hwfs,fol,sericite-quartz schist; light gray; strong fol...,NaN,NaN
370217,NaN,KM-21-33,553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,RH,hwfs,fol,sericite-quartz schist; light gray; strong fol...,NaN,NaN


In [12]:
merged=merged.groupby(['hole_id','true_depth']).progress_apply(fill_merge_groups).reset_index(drop=True)
merged

  0%|          | 0/332955 [00:00<?, ?it/s]

,sample_id,hole_id,true_depth,geo_x,instrument serial num,reading #,date,time,method name,test label,...,angle,description_structure,"/.,",118,geo_lithology,lithology,texture,description_lithology,fol,geo_2
0,NaN,KM-20-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DL,r,NaN,No recovery.,NaN,NaN
1,NaN,KM-20-01,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DL,r,NaN,No recovery.,NaN,NaN
2,NaN,KM-20-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DL,r,NaN,No recovery.,NaN,NaN
3,NaN,KM-20-01,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DL,r,NaN,No recovery.,NaN,NaN
4,NaN,KM-20-01,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DL,r,NaN,No recovery.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347053,NaN,Pre 2022,2834.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347054,NaN,Pre 2022,2835.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347055,NaN,Pre 2022,2835.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347056,NaN,Pre 2022,2836.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
merged['u_id']=merged.fillna(np.inf).groupby(merged.drop('true_depth',axis=1).columns.to_list()).ngroup()

In [40]:
merged['from_ft']=np.inf
merged['to_ft']=np.inf
merged

,sample_id,hole_id,true_depth,geo_x,instrument serial num,reading #,date,time,method name,test label,...,118,geo_lithology,lithology,texture,description_lithology,fol,geo_2,u_id,from_ft,to_ft
0,NaN,KM-20-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DL,r,NaN,No recovery.,NaN,NaN,27,inf,inf
1,NaN,KM-20-01,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DL,r,NaN,No recovery.,NaN,NaN,27,inf,inf
2,NaN,KM-20-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DL,r,NaN,No recovery.,NaN,NaN,27,inf,inf
3,NaN,KM-20-01,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DL,r,NaN,No recovery.,NaN,NaN,27,inf,inf
4,NaN,KM-20-01,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DL,r,NaN,No recovery.,NaN,NaN,27,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347053,NaN,Pre 2022,2834.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6375,inf,inf
347054,NaN,Pre 2022,2835.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6375,inf,inf
347055,NaN,Pre 2022,2835.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6375,inf,inf
347056,NaN,Pre 2022,2836.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6375,inf,inf


In [121]:
merged_test=merged.groupby(merged['u_id']).progress_apply(pull_start_end).reset_index(drop=True)

  0%|          | 0/10868 [00:00<?, ?it/s]

In [122]:
merged_test.drop('true_depth',axis=1, inplace=True)

In [123]:
final_cols=['sample_id','hole_id','from_ft','to_ft']
des_cols = merged_test.filter(like='description').columns
other_cols= [col for col in merged_test.columns if col not in final_cols]
merged_test=pd.concat([merged_test[final_cols],merged_test[des_cols],merged_test[other_cols]],axis=1)
merged_test

,sample_id,hole_id,from_ft,to_ft,description_x,description_y,description_mineralization,description_structure,description_lithology,geo_x,...,description_structure,"/.,",118,geo_lithology,lithology,texture,description_lithology,fol,geo_2,u_id
0,NaN,KM-20-01,913.2,963.2,NaN,NaN,NaN,NaN,Quartz-sericite schist.,NaN,...,NaN,NaN,NaN,DL,fwfs,fol,Quartz-sericite schist.,NaN,NaN,0
1,NaN,KM-20-01,977.2,1099.7,NaN,NaN,NaN,NaN,Quartz-sericitie schist. Strongly fractured an...,NaN,...,NaN,NaN,NaN,DL,fwfs,fol,Quartz-sericitie schist. Strongly fractured an...,NaN,NaN,1
2,NaN,KM-20-01,705.8,712.8,NaN,NaN,NaN,NaN,Quartz-sericite schist(phyllite). Very fine-gr...,NaN,...,NaN,NaN,NaN,DL,hwfs,fol,Quartz-sericite schist(phyllite). Very fine-gr...,NaN,NaN,2
3,NaN,KM-20-01,885.4,904.4,NaN,NaN,NaN,NaN,Sericite-quartz schist. Grey.,NaN,...,NaN,NaN,NaN,DL,hwfs,fol,Sericite-quartz schist. Grey.,NaN,NaN,3
4,NaN,KM-20-01,764.7,845.7,NaN,NaN,NaN,NaN,"Graph-ser-qtz schist, graph-qtz-ser schist and...",NaN,...,NaN,NaN,NaN,DL,hwgs,fol,"Graph-ser-qtz schist, graph-qtz-ser schist and...",NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10863,G453727,KM-22-79,2271.0,2275.0,sericite-quartz schist; 1%py,felsic schist; sparse pyrite stringers,sericite-quartz schist; 1%py,NaN,sericite-quartz-chlorite schist; variable grey...,JR,...,NaN,NaN,NaN,JV,fwfs,fol,sericite-quartz-chlorite schist; variable grey...,NaN,NaN,10863
10864,G453728,KM-22-79,2275.5,2278.5,sericite-quartz schist; 1%py,felsic schist; sparse pyrite stringers,sericite-quartz schist; 1%py,NaN,sericite-quartz-chlorite schist; variable grey...,JR,...,NaN,NaN,NaN,JV,fwfs,fol,sericite-quartz-chlorite schist; variable grey...,NaN,NaN,10864
10865,NOTE,KM-20-09,2007.0,2008.5,NO CORE - NO SAMPLE,"intense qt-ser alt, soft",2 % fine py along laminations,NaN,"ser-qtz schist; highly variable, zones of whit...",RH,...,NaN,NaN,NaN,RH,hwfs,fol,"ser-qtz schist; highly variable, zones of whit...",NaN,NaN,10865
10866,x x x,KM-20-14A,1364.0,1365.0,NO RECOVERY - no sample,intense sericite/mic,NaN,NaN,ore zone - details in sample description log,RH,...,NaN,NaN,NaN,RH,sm-ms,mas,ore zone - details in sample description log,NaN,NaN,10866


In [124]:
merged_test=merged_test.T.drop_duplicates(keep='first').T
merged_test

,sample_id,hole_id,from_ft,to_ft,description_x,description_y,description_mineralization,description_structure,description_lithology,geo_x,...,geo_structure,structure,angle,"/.,",geo_lithology,lithology,texture,fol,geo_2,u_id
0,NaN,KM-20-01,913.2,963.2,NaN,NaN,NaN,NaN,Quartz-sericite schist.,NaN,...,NaN,NaN,NaN,NaN,DL,fwfs,fol,NaN,NaN,0
1,NaN,KM-20-01,977.2,1099.7,NaN,NaN,NaN,NaN,Quartz-sericitie schist. Strongly fractured an...,NaN,...,NaN,NaN,NaN,NaN,DL,fwfs,fol,NaN,NaN,1
2,NaN,KM-20-01,705.8,712.8,NaN,NaN,NaN,NaN,Quartz-sericite schist(phyllite). Very fine-gr...,NaN,...,NaN,NaN,NaN,NaN,DL,hwfs,fol,NaN,NaN,2
3,NaN,KM-20-01,885.4,904.4,NaN,NaN,NaN,NaN,Sericite-quartz schist. Grey.,NaN,...,NaN,NaN,NaN,NaN,DL,hwfs,fol,NaN,NaN,3
4,NaN,KM-20-01,764.7,845.7,NaN,NaN,NaN,NaN,"Graph-ser-qtz schist, graph-qtz-ser schist and...",NaN,...,NaN,NaN,NaN,NaN,DL,hwgs,fol,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10863,G453727,KM-22-79,2271.0,2275.0,sericite-quartz schist; 1%py,felsic schist; sparse pyrite stringers,sericite-quartz schist; 1%py,NaN,sericite-quartz-chlorite schist; variable grey...,JR,...,NaN,NaN,NaN,NaN,JV,fwfs,fol,NaN,NaN,10863
10864,G453728,KM-22-79,2275.5,2278.5,sericite-quartz schist; 1%py,felsic schist; sparse pyrite stringers,sericite-quartz schist; 1%py,NaN,sericite-quartz-chlorite schist; variable grey...,JR,...,NaN,NaN,NaN,NaN,JV,fwfs,fol,NaN,NaN,10864
10865,NOTE,KM-20-09,2007.0,2008.5,NO CORE - NO SAMPLE,"intense qt-ser alt, soft",2 % fine py along laminations,NaN,"ser-qtz schist; highly variable, zones of whit...",RH,...,NaN,NaN,NaN,NaN,RH,hwfs,fol,NaN,NaN,10865
10866,x x x,KM-20-14A,1364.0,1365.0,NO RECOVERY - no sample,intense sericite/mic,NaN,NaN,ore zone - details in sample description log,RH,...,NaN,NaN,NaN,NaN,RH,sm-ms,mas,NaN,NaN,10866


In [125]:

for col in merged_test.columns:
    if merged_test[col].isna().sum()==len(merged_test):
        print(f'drop {col}: no data')
        merged_test=merged_test.drop(col,axis=1)
merged_test=merged_test.sort_values(by=['sample_id','hole_id','from_ft'],ascending=False)
merged_test=merged_test.set_index('sample_id')
merged_test

drop instrument serial num: no data


,hole_id,from_ft,to_ft,description_x,description_y,description_mineralization,description_structure,description_lithology,geo_x,work_order,...,geo_structure,structure,angle,"/.,",geo_lithology,lithology,texture,fol,geo_2,u_id
sample_id,,,,,,,,,,,,,,,,,,,,,
x x x,KM-20-16,1620.0,1625.5,"VOID - mine workings, no sample",NaN,NaN,NaN,ore zone - detailed in mineralization and samp...,RH,TU20177222,...,NaN,NaN,NaN,NaN,RH,sm-ms,mas,NaN,NaN,10867
x x x,KM-20-14A,1364.0,1365.0,NO RECOVERY - no sample,intense sericite/mic,NaN,NaN,ore zone - details in sample description log,RH,TU20177222,...,NaN,NaN,NaN,NaN,RH,sm-ms,mas,NaN,NaN,10866
NOTE,KM-20-09,2007.0,2008.5,NO CORE - NO SAMPLE,"intense qt-ser alt, soft",2 % fine py along laminations,NaN,"ser-qtz schist; highly variable, zones of whit...",RH,NaN,...,NaN,NaN,NaN,NaN,RH,hwfs,fol,NaN,NaN,10865
G453728,KM-22-79,2275.5,2278.5,sericite-quartz schist; 1%py,felsic schist; sparse pyrite stringers,sericite-quartz schist; 1%py,NaN,sericite-quartz-chlorite schist; variable grey...,JR,NaN,...,NaN,NaN,NaN,NaN,JV,fwfs,fol,NaN,NaN,10864
G453727,KM-22-79,2271.0,2275.0,sericite-quartz schist; 1%py,felsic schist; sparse pyrite stringers,sericite-quartz schist; 1%py,NaN,sericite-quartz-chlorite schist; variable grey...,JR,NaN,...,NaN,NaN,NaN,NaN,JV,fwfs,fol,NaN,NaN,10863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,KM-20-01,729.0,744.5,NaN,Tan to light brn coloured mica.,NaN,NaN,Feldspar-sericite-quart schist. Locally weakly...,NaN,NaN,...,NaN,NaN,NaN,NaN,DL,hwfs,fol,NaN,NaN,65
NaN,KM-20-01,828.0,842.0,NaN,Pervasive sericite alteration with quartz in v...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66
NaN,KM-20-01,830.0,841.0,NaN,Pervasive sericite alteration with quartz in v...,NaN,"Strongly fractured, locally crushed with gouge.",NaN,NaN,NaN,...,DL,frac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67


In [126]:
merged_test.to_excel('/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/master_MASTER.xlsx')